# Block Ciphers

Block ciphers are encryption algorithms that work on fixed-length chunks (blocks) of binary data. We divide the plaintext into fixed-length blocks and feed them into a function (along with the key) that encrypts the block into a ciphertext block. For each block B, we encrypt part of the message $ciphertext_B=E(key,plaintext_B)$. Repeating this process, we can encrypt the entire message block by block.

Decryption works similarly. We decrypt the ciphertext block by block. $plaintext_B=D(key,ciphertext_B)$. Importantly, this means the encryption function must have an inverse. That inverse is the decryption function. Furthermore, the pair of encryption and decryption functions must only work when the correct key is supplied. As you can imagine, designing a block cipher is quite difficult.

## AES

Advanced Encryption Standard is a block cipher standardized by the NIST in 2001. If you want to securely transmit a message today, AES is the tool for the job. Let's learn how we can encrypt messages in Python using AES.

To import the library for AES, we need to import it from the Cipher submodule of the Crypto module. Importing all those libraries would be a bit much. Fortunately, Python lets us import just the part of a library we plan to use with the **from** keyword.

In [ ]:
from Crypto.Cipher import AES

We can now set up an AES cipher system by calling the "new" function on a key. I generated a 16-byte random key using random.org

In [ ]:
from Crypto.Cipher import AES
key="cPh0zi4Za48OvmUx".encode('ASCII')
cipher=AES.new(key)

We can then use this AES cipher to encrypt a message.

In [ ]:
plaintext="MySecretMessage!"
ciphertext=cipher.encrypt(plaintext.encode('ASCII'))
print(ciphertext)

We can also use it to decrypt the message.

In [ ]:
decoded=cipher.decrypt(ciphertext).decode('ASCII')
print(decoded)

Let's encode another message.

In [ ]:
plaintext="Trying Again"
ciphertext=cipher.encrypt(plaintext.encode('ASCII'))
print(ciphertext)

Uh oh... what happened?

## Padding

The problem in the previous section is that the length of our message wasn't a multiple of 16 bytes. In other words, our message was slightly shorter than a block. To correct for this, we need to **pad** the message (add a little bit extra.) The standard way to do this is to figure out how many bytes of padding are needed, encode that number in a byte, and then repeat that byte to pad out the message (e.g. if we need twelve bytes of padding, repeat the number 12 twelve times.)

Unfortunately, there's no built-in padding function. Let's write our own padding function. If the message is $n$ bytes long and the block size is $b$ bytes, how many bytes do we need to pad? The message will fit into $\frac{n}{b}$ blocks with some remainder. How do we find the remainder? Our old pal modulo $n \bmod b$. Our message fills those blocks. How many blocks are left over? $b - n \bmod b$.

Since we know the padding has to be between 0 and 15, we can encode the pad length as a single hex character.

In [ ]:
def pad(message):
    blockSize=16
    padLen=blockSize-len(message)%blockSize
    padChar=hex(padLen)[-1]
    return message+padChar*padLen
    
print(pad("Hello"))

Now let's write a function to unpad a message. All we need to do is figure out how many characters were used to pad the message and then slice off the padding characters. There's just one complication. Our padding function **always** pads the message, even when the message  length is a multiple of 16 bytes. You might not have noticed, but you can go back and try it out for yourself.

This is a good behavior. We should always pad, regardless of the message length. If we don't, a person who receives a message has no way of knowing if we intended to send "Hello" or "Hellobbbbbbbbbbb". Did we pad the message or not? They can't tell. If we always pad, there's no ambiguity.

This is all to say that when the padding character is 0, we should remove 16 bytes of padding.

In [ ]:
def unpad(message):
    blockSize=16
    padLen=int(message[-1],16)
    if(padLen==0):
        padLen=16
    return message[:-padLen]

padded=pad("TESTINGTESTINGTE")
print(padded)
print(unpad(padded))

Now we can encrypt any message with AES. All we need to do is pad it first.

In [ ]:
from Crypto.Cipher import AES
key="cPh0zi4Za48OvmUx".encode('ASCII')
cipher=AES.new(key)

plaintext="THIS MESSAGE IS NOT A MULTIPLE OF 16 BYTES"
print("Plaintext:\t",plaintext)
padded=pad(plaintext)
print("Padded:\t\t",padded)
ciphertext=cipher.encrypt(padded.encode('ASCII'))
print("Ciphertext:\t",ciphertext)
decoded=cipher.decrypt(ciphertext).decode('ASCII')
print("Decoded:\t",decoded)
unpadded=unpad(decoded)
print("Unpadded:\t",unpadded)

I'm going to demonstrate how we could use AES in practice. We're going to encrypt a file. This picture.
![cat](cat.JPG)
Some of the code I'm using to load the image is a bit advanced. Don't worry if you don't quite understand how it works. This is just a demonstration. In the end, I'm just loading the image data into a string of bytes of plaintext and then encrypting them.

In [ ]:
from io import BytesIO
from PIL import Image
from Crypto.Cipher import AES
import base64
from IPython.display import display

image = Image.open("cat.JPG")
size=image.size
mode=image.mode

plaintext=image.tobytes()
plaintext=plaintext+b'0'*(16-len(plaintext)%16)
key="cPh0zi4Za48OvmUx".encode('ASCII')
cipher=AES.new(key)
ciphertext=cipher.encrypt(plaintext)

The image data is now encrypted. Let's do something strange. We're going to view the ciphertext data as an image. We **should** just see the total nonsense of random noise. This is ciphertext. Nobody should be able to learn **anything** about the input.

In [ ]:
image2=Image.frombytes(mode,size,ciphertext)
display(image2)

Uh oh. Some information from the original image is left over. How did this happen? We're using "military grade encryption" here! What's going on?

## Modes of Operation

In the previous example, we encrypted each block of the ciphertext using the same key. This means identical plaintext blocks are encrypted to identical ciphertext blocks. As a result, some of the shapes in the image remain visible.

This way of operating a block cipher is known as Electronic Code Book (ECB). As you can see, it is not secure. We need a way of operating a block cipher that encrypts each plaintext block differently, even when the plaintext is identical. Fortunately, there are other modes of operation that are more secure.

Cipher Block Chaining (CBC) is a mode of operation that XORs each plaintext block with the previous encrypted block. This mode is secure.

In [ ]:
from io import BytesIO
from PIL import Image
from Crypto.Cipher import AES
import base64
from IPython.display import display

image = Image.open("cat.JPG")
size=image.size
mode=image.mode

plaintext=image.tobytes()
plaintext=plaintext+b'0'*(16-len(plaintext)%16)
key="cPh0zi4Za48OvmUx".encode('ASCII')
iv="W3O53gzgWmJUKoOF"
cipher=AES.new(key,AES.MODE_CBC,iv)
ciphertext=cipher.encrypt(plaintext)
image2=Image.frombytes(mode,size,ciphertext)
display(image2)

## Exercises
Below is the only exercise for this lesson. You're using real encryption here, so be patient. The libraries are very picky. Don't be afraid to ask for help if you need it.

1) This message was encrypted using AES in CBC with key "7TauhZcYVbEPS6ds" and initialization vector "rEXcRynoUxGCc32I". Decrypt it.

In [ ]:
key="7TauhZcYVbEPS6ds"
iv="rEXcRynoUxGCc32I"
ciphertext=b'!\xb3\xb6\xe2\xee\r\x7f\x89\x9d\xb4a]\xd2,\x0c^\x1ea\xf7\x0b\x92\xbf\t\xbbG\xebahu14\x8b|\x98Dc\xd0x\xb1*\x88d\x83\xacM\xd51_\xcb?\xfa\x08\xce\x84\xecJt\x14\x1a\x17<\x13;(\x07\xf6\xceP\x17\x1cvA\x9fX\x8a\x9a\xae6\x88!.^>\xaf\xd4Q\xd1N\x17\xe5\x003\'\xf4\xc8\x19/\xa3AD\x9d\x1e\xcfy\xd3\xe05\x0f\xc9\xe9\xc1\xba\x03\x90\xc2\x02\xa0\xa0\x00\xe9\x1d\x7f\xc0\xee,\xbe\xbe\x8e\xb8[\xd2\xafBZ\xc6V\x8f\xc9Z\'\xcf\x9dt>\x99)\xa8\x89\xa8\xa6\xda\xd2q>\x95P\x1b\x7f\xa1\xad\xa5\xcb9\xdf\x80\xe9\x17\x10\'\xd4\xd2.H\x97S?\x1e\xf1\xfa=\r\x8a\xf7_\xf4K\xd5aI\xad\x8eCr\xd6\xf9\xb3l\x1d|\x1b\xc8.x\xb08\xf1\xb7\xc2]\x13\xabD\xfe\x16\xc4\xa3-\xe4\xa3`\x88\xe4#\xb8\x91\xef\x1c\xab\xcf|1\x10\x1f\x10+\xe9\x1c\xf8\xa0\xc8\x11]\xbc(\'()q\xf8\x88\x90\x10*\x1dJ\x86\xa8\xc1;\xab\x7f\xe9\x1a\xf1\x81\xf5\x14\x04\xc5\x9b\x9f\xd4\x9eG\x06\xc8\xbf/\xbc\xd8g\xdd\xcd\x82\xad\xb0[\x7f\x15\x8b`j\xb7\xb6\x1f4\xc7\xf1y\xc5?a\xe0R0\xd7o\xf5\x9b~\x1b\x90t\xb3\xeb\x9f\xcas\xa4U\xc1\x97\xfb\xed\x89\xfd\x17S\x0b\x1a\xd0!\x8c3_>\x0e\x07sC\x9a\x98\xec\x8c\xd7\x15P\xa3B\x86`\xa5\xdcja\xe1d\x85\xfb\xd0Q\x0c\x9b\x98x%\x94\x83\x03\xce\x08\xcdk\xde\x98b4Z\x01\x82\x0bc\xaf\xdf\xf3\xc2"\xa2\xfa\x82j3I\xc7\x0b\xe9Q\x15;\xa5#0H\x89m\x0e,0\xc8\x9fC\xed\xc1A\xabJj\xbc\xf3+\x08\xdf\x04q\x153\xe6\xd7\xecu\x12\xa2\x1d\xea\x12\xc27\x01)N\xfdT\x86\xe01\xc1\xd2\xde\x96%\xc5K:gGI*\xa3S\xd9\xb3\xa9\xc8/\x00Kd\xb7\x085\xc5y\x13\xc3\xed\xad-\x12S\xbd\x08{\xdc\x8f\x99D\xdc\xdf5p^\xbcq|F^O\x83G\x1dgO.u\x99\xd3^M\xee\xda\xb2\xf0\xbc\xcc\xb4\x01\xc6\xbb\xd7@\xf9\x1cNx&\x14\x1f\xd48F\x96\x8c&\x86\xee\'\xc9\x87\x02 N\xb5a\xcb'